# Assignment 2: Building a Simple Index

In this assignment, we will build a simple search index, which we will use later for Boolean retrieval. The assignment tasks are again at the bottom of this document.

## Loading the Data

In [1]:
Summaries_file = 'data/pandemic_Summaries.pkl.bz2'
Abstracts_file = 'data/pandemic_Abstracts.pkl.bz2'

In [2]:

import pickle, bz2
from collections import namedtuple

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )

for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )

Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

Let's have a look at what the data looks like for an example of a paper:

In [4]:
Summaries[293742]

paper(title='Evolution of human influenza A viruses in nature: recombination contributes to genetic variation of H1N1 strains.', authors=['Young JF', 'Palese P'], year=1979, doi='10.1073/pnas.76.12.6547')

In [5]:
Abstracts[293742]

'In June of 1977, a new influenza A pandemic was started by strains of the H1N1 serotype. Oligonucleotide fingerprint analysis of the RNA from viruses isolated during the early stage of this pandemic demonstrated that genetic variation among these 1977 strains could be attributed to sequential mutation [Young, J.F., Desselberger, U. & Palese, P. (1979) Cell, 18, 73-83]. Examination of more recent strains revealed that the H1N1 variants that were isolated in the winter of 1978-1979 differed considerably from the H1N1 viruses isolated the previous year. Oligonucleotide and peptide map analysis of the new prototype strain (A/Cal/10/78) suggested that it arose by recombination. It appears that only the HA, NA, M, and NS genes of this virus are derived from the earlier H1N1 viruses and that the P1, P2, P3, and NP genes most likely originate from an H3N2 parent. These data suggest that genetic variation in influenza virus strains of the same serotype is not restricted to mutation alone, but 

## Some Utility Functions

We'll define some utility functions that allow us to tokenize a string into terms, perform linguistic preprocessing on a list of terms, as well as a function to display information about a paper in a nice way. Note that these tokenization and preprocessing functions are rather naive. We will improve them in a later assignment.

In [6]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on the list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

print(preprocess(tokenize("Recognition of the positive effects of music and dance for mental health.")))





['recognition', 'of', 'the', 'positive', 'effects', 'of', 'music', 'and', 'dance', 'for', 'mental', 'health.']


In [7]:
from IPython.display import display, HTML
import re

def display_summary( id, show_abstract=False, show_id=True, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long author lists, and adds a link to the paper's DOI (when available).
    """
    s = Summaries[id]
    lines = []
    title = s.title
    if s.doi != '':
        title = '<a href=http://dx.doi.org/{:s}>{:s}</a>'.format(s.doi, title)
    title = '<strong>' + title + '</strong>'
    lines.append(title)
    authors = ', '.join( s.authors[:20] ) + ('' if len(s.authors) <= 20 else ', ...')
    lines.append(str(s.year) + '. ' + authors)
    if (show_abstract):
        lines.append('<small><strong>Abstract:</strong> <em>{:s}</em></small>'.format(Abstracts[id]))
    if (show_id):
        lines.append('[ID: {:d}]'.format(id))
    if (extra_text != ''):
         lines.append(extra_text)
    display( HTML('<br>'.join(lines)) )

display_summary(293742, show_abstract=True)

## Creating our first index

We will now create an _inverted index_ based on the words in the titles and abstracts of the papers in our dataset. We will implement our inverted index as a Python dictionary with term strings as keys and posting lists (implemented as Python lists) as values. We include all the tokens we can find in the title and (if available) in the abstract:

In [9]:
from collections import defaultdict

inverted_index = defaultdict(list)

# This can take a minute:
for id in sorted(Summaries.keys()):
    term_set = set(preprocess(tokenize(Summaries[id].title)))
    if id in Abstracts:
        term_set.update(preprocess(tokenize(Abstracts[id])))
    for term in term_set:
        inverted_index[term].append(id)

Let's see what's in the index for the example term 'ivermectin':

In [10]:
print(inverted_index['ivermectin'])

[32293834, 32462282, 32611256, 32752944, 32863755, 33246694, 33721651, 33748818, 34395311, 34428682, 35545850, 35857778, 36068517, 36214394, 36374569, 37692571, 38371720, 38844670, 39070952, 39458594, 39672345, 39892677, 40598561]


We can now use this inverted index to answer simple one-word queries, for example to show all papers that contain the word 'ivermectin':

In [11]:
query_word = 'ivermectin'
for i in inverted_index[query_word]:
    display_summary(i, show_abstract=True)

----------

# Tasks

**Your name:** ...

### Task 1

Implement the function `sorted_merge_and` outlined below. This function takes two posting lists from the index that can be assumed to be sorted already (e.g. [3,5,8,11,18] and [5,7,8,9,11,12]), and it should return the result of the merging of the two lists with AND. The resulting list should therefore include all the elements that appear in both lists. As explained on the slides, this operation should take advantage of the input lists being sorted already, should not perform any additional sorting operation, and should go through each of the input lists just once. Then, test your function with an example.

In [ ]:
#your code here

### Task 2

Similarly as above, implement the function `sorted_or` outlined below that executes an OR merging of the lists. The resulting list should therefore include all the elements that appear in at least one of the lists. Again, this operation should take advantage of the input lists being sorted already, should not perform any additional sorting operation, and should go through each of the input lists just once. Elements that appear in both input list should only appear once in the output list. Test your function again with an example.

In [46]:
#your code here

### Task 3

Construct a function called `and_query` that takes as input a single string, consisting of one or more words, and returns as function value a list of matching documents. `and_query`, as its name suggests, should require that all query terms are present in the documents of the result list.

For that, access the variable `inverted_index` from above and use the method `merge_and` that you defined. Also use the `tokenize` and `preprocess` functions we defined above to tokenize and preprocess your query.

Again demonstrate the working of your function with an example (choose one that leads to fewer than 100 hits to not overblow this notebook file).

In [47]:
#your code here

### Task 4

Construct another function called `query_or` that works in the same way as `query_and` you just implemented, but returns as function value the documents that contain _at least one_ of the words in the query, using the `sorted_merge_or` function you defined.

Demonstrate the working of this function also with an example (again, choose one that leads to fewer than 100 hits).

In [48]:
#your code here

### Task 5

Why does `query_and('ivermectin vitro')` not return paper '34428682' and '39892677', even though it mentions these terms in the title and abstract? (You do not have to implement anything to fix this yet!)

**Answer:** 

# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done **individually**, and that code sharing or copying are **strictly forbidden** and will be punished.